# 运行试验

可以使用 Azure 机器学习 SDK 运行记录指标并生成输出的代码试验。这是 Azure 机器学习中大多数机器学习操作的核心。

## 连接到工作区

所有试验和相关资源都在 Azure 机器学习工作区中进行管理。大多数情况下，应该将工作区配置存储在 JSON 配置文件中。这会使重新连接变得更加容易，且无需记住 Azure 订阅 ID 等详细信息。可以从 Azure 门户的工作区中的边栏选项卡下载 JSON 配置文件，但如果在工作区中使用的是计算实例，则该配置文件已下载到根文件夹中。

以下代码使用配置文件连接到工作区。

> **备注**：如果尚未与 Azure 订阅建立经过身份验证的会话，则系统将提示你通过执行以下操作进行身份验证：单击链接，输入验证码，然后登录到 Azure。

In [ ]:
import azureml.core
from azureml.core import Workspace

# Load the workspace from the saved config file
ws = Workspace.from_config()
print('Ready to use Azure ML {} to work with {}'.format(azureml.core.VERSION, ws.name))

## 运行试验

数据科学家需执行的一项最基本任务是创建并运行可处理和分析数据的试验。在本练习中，你将学习如何使用 Azure ML 试验运行 Python 代码并记录从数据中提取的值。这种情况下，你将使用一个简单数据集，其中包含已进行糖尿病检测的患者的详细信息。你将运行试验以探索数据、提取统计信息、可视化效果和数据样本。你将使用的大多数代码是相当通用的 Python，就像你在任何数据探索过程中运行的那样。但需额外增加几行代码以使用 Azure ML 试验记录运行的详细信息。

In [ ]:
from azureml.core import Experiment
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline 

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name="mslearn-diabetes")

# Start logging data from the experiment, obtaining a reference to the experiment run
run = experiment.start_logging()
print("Starting experiment:", experiment.name)

# load the data from a local file
data = pd.read_csv('data/diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Plot and log the count of diabetic vs non-diabetic patients
diabetic_counts = data['Diabetic'].value_counts()
fig = plt.figure(figsize=(6,6))
ax = fig.gca()    
diabetic_counts.plot.bar(ax = ax) 
ax.set_title('Patients with Diabetes') 
ax.set_xlabel('Diagnosis') 
ax.set_ylabel('Patients')
plt.show()
run.log_image(name='label distribution', plot=fig)

# log distinct pregnancy counts
pregnancies = data.Pregnancies.unique()
run.log_list('pregnancy categories', pregnancies)

# Log summary statistics for numeric columns
med_columns = ['PlasmaGlucose', 'DiastolicBloodPressure', 'TricepsThickness', 'SerumInsulin', 'BMI']
summary_stats = data[med_columns].describe().to_dict()
for col in summary_stats:
    keys = list(summary_stats[col].keys())
    values = list(summary_stats[col].values())
    for index in range(len(keys)):
        run.log_row(col, stat=keys[index], value = values[index])
        
# Save a sample of the data and upload it to the experiment output
data.sample(100).to_csv('sample.csv', index=False, header=True)
run.upload_file(name='outputs/sample.csv', path_or_stream='./sample.csv')

# Complete the run
run.complete()

## 查看运行详细信息

在 Jupyter Notebook 中，可以使用“**RunDetails**”小组件查看运行详细信息的可视化效果。

In [ ]:
from azureml.widgets import RunDetails

RunDetails(run).show()

### 在 Azure 机器学习工作室中查看更多详细信息

请注意，“**RunDetails**”小组件包含一个链接，用于查看 Azure 机器学习工作室中的**运行详细信息**。单击此链接，打开包含运行详细信息的新浏览器标签页（也可以直接打开 [Azure 机器学习工作室](https://ml.azure.com)，在“**试验**”页面中查找运行）。在 Azure 机器学习工作室中查看运行时，请注意以下内容：

- “**详细信息**”选项卡包含试验运行的常规属性。
- “**指标**”选项卡使你可以选择记录的指标并以表格或图表形式查看。
- “**图像**”选项卡使你可以选择和查看试验中记录的所有图像或绘图（本例中为“*标签分布*”绘图）
- “**子运行**”选项卡列出所有子运行（本试验中没有）。
- “**输出 + 日志**”选项卡显示试验生成的输出或日志文件。
- “**快照**”选项卡包含运行试验代码的文件夹中的所有文件（本例中，所有文件与此笔记本位于同一文件夹）。
- “**说明**”选项卡用于显示试验生成的模型说明（本例中没有）。
- “**公平性**”选项卡用于可视化预测性能差异，以帮助你评估机器学习模型的公平性（本例中没有）。

### 使用 SDK 检索试验详细信息

先前运行的代码中的 **run** 变量是 **Run** 对象的实例，它是对 Azure 机器学习中试验的单个运行的引用。可以使用此引用来获取有关运行及其输出的信息：

In [ ]:
import json

# Get logged metrics
print("Metrics:")
metrics = run.get_metrics()
for metric_name in metrics:
    print(metric_name, ":", metrics[metric_name])

# Get output files
print("\nFiles:")
files = run.get_file_names()
for file in files:
    print(file)

可以使用 **download_file** 方法单独下载试验生成的文件，也可以使用 **download_files** 方法检索多个文件。以下代码下载运行的 **output** 文件夹中的所有文件：

In [ ]:
import os

download_folder = 'downloaded-files'

# Download files in the "outputs" folder
run.download_files(prefix='outputs', output_directory=download_folder)

# Verify the files have been downloaded
for root, directories, filenames in os.walk(download_folder): 
    for filename in filenames:  
        print (os.path.join(root,filename))

如果需要对试验运行进行故障排除，则可以使用 **get_details** 方法检索有关运行的基本详细信息，也可以使用 **get_details_with_logs** 方法检索运行详细信息以及运行期间生成的日志文件的内容：

In [ ]:
run.get_details_with_logs()

请注意，详细信息包括与运行试验的计算目标有关的信息，有其开始以及结束的日期和时间。此外，因为包含试验代码的笔记本（此笔记本）在克隆的 Git 存储库中，所以存储库、分支以及状态的相关详细信息都记录在运行历史记录中。

在本例中，请注意，详细信息中的 **logFiles** 条目指示未生成任何日志文件。这对于内联试验（例如你所运行的内联试验）十分典型，但当你以试验方式运行脚本时，会变得更加有趣，接下来我们将介绍这一内容。

## 运行试验脚本

上一示例中，你在此笔记本中内联运行了一个试验。更为灵活的解决方案如下：为试验创建单独的脚本，将其与其所需的其他所有文件一同存储在一个文件夹中，然后使用 Azure ML 基于文件夹中的脚本运行试验。

首先，为试验文件创建文件夹，然后将数据复制到其中：

In [ ]:
import os, shutil

# Create a folder for the experiment files
folder_name = 'diabetes-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(folder_name, "diabetes.csv"))

接下来，创建包含试验代码的 Python 脚本，然后将其保存到试验文件夹。

> **备注**：运行以下单元格只创建脚本文件，不运行文件！

In [ ]:
%%writefile $folder_name/diabetes_experiment.py
from azureml.core import Run
import pandas as pd
import os

# Get the experiment run context
run = Run.get_context()

# load the diabetes dataset
data = pd.read_csv('diabetes.csv')

# Count the rows and log the result
row_count = (len(data))
run.log('observations', row_count)
print('Analyzing {} rows of data'.format(row_count))

# Count and log the label counts
diabetic_counts = data['Diabetic'].value_counts()
print(diabetic_counts)
for k, v in diabetic_counts.items():
    run.log('Label:' + str(k), v)
      
# Save a sample of the data in the outputs folder (which gets uploaded automatically)
os.makedirs('outputs', exist_ok=True)
data.sample(100).to_csv("outputs/sample.csv", index=False, header=True)

# Complete the run
run.complete()

此代码是之前使用的内联代码的简化版本。但请注意以下内容：
- 运行脚本时，它使用 `Run.get_context()` 方法检索试验运行上下文。
- 它从脚本所在文件夹加载糖尿病数据。
- 它创建名为 **outputs** 的文件夹，并将示例文件写入其中 - 该文件夹会自动上传到试验运行

现在基本可以运行试验。要运行脚本，必须创建一个“**ScriptRunConfig**”命令以标识要在试验中运行的 Python 脚本文件，然后基于该脚本运行试验。

> **备注**：ScriptRunConfig 还可以确定计算目标和 Python 环境。在本例中，Python 环境被定义为包括一些 Conda 和 pip 包，但省略了计算目标，因此将使用默认的本地计算。

以下单元格配置并提交基于脚本的试验。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.widgets import RunDetails

# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Create a script config
script_config = ScriptRunConfig(source_directory=experiment_folder,
                                script='diabetes_experiment.py',
                                environment=env)

# submit the experiment
experiment = Experiment(workspace=ws, name='mslearn-diabetes')
run = experiment.submit(config=script_config)
RunDetails(run).show()
run.wait_for_completion()

和以前一样，可以使用小组件或指向 [Azure 机器学习工作室](https://ml.azure.com)中的试验的链接来查看试验生成的输出，还可以编写代码来检索其生成的指标和文件：

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
print('\n')
for file in run.get_file_names():
    print(file)

请注意，这次，运行生成了一些日志文件。可以在窗口小组件中查看这些内容，也可以照常使用 **get_details_with_logs** 方法，只不过这次输出将包含日志数据。

In [ ]:
run.get_details_with_logs()

虽然可以在上面的输出中查看日志详细信息，但通常更容易下载日志文件并在文本编辑器中查看它们。

In [ ]:
import os

log_folder = 'downloaded-logs'

# Download all files
run.get_all_logs(destination=log_folder)

# Verify the files have been downloaded
for root, directories, filenames in os.walk(log_folder): 
    for filename in filenames:  
        print (os.path.join(root,filename))

## 查看试验运行历史记录

多次运行同一试验后，可以在 [Azure 机器学习工作室](https://ml.azure.com)中查看历史记录并浏览所记录的每个运行。或者，可以从工作区按名称检索试验，然后使用 SDK 遍历其运行：

In [ ]:
from azureml.core import Experiment, Run

diabetes_experiment = ws.experiments['mslearn-diabetes']
for logged_run in diabetes_experiment.get_runs():
    print('Run ID:', logged_run.id)
    metrics = logged_run.get_metrics()
    for key in metrics.keys():
        print('-', key, metrics.get(key))

## 使用 MLflow

MLflow 是用于管理机器学习进程的开放源代码平台。在 Databricks 环境中通常（但不是唯一）使用它来协调试验和跟踪指标。在 Azure 机器学习试验中，你可以使用 MLflow 来跟踪指标，以作为原生日志功能的替代方法。

要利用此功能，你将需要 **azureml-mlflow** 包，所以我们要确认已将其安装。

In [ ]:
!pip show azureml-mlflow

### 将 MLflow 与内联试验一起使用

要使用 MLflow 跟踪内联试验的指标，必须将 MLflow 的“*跟踪 URI*”设置到运行试验的工作区。这使你可以使用“**mlflow**”跟踪方法将数据记录到试验运行中。

In [ ]:
from azureml.core import Experiment
import pandas as pd
import mlflow

# Set the MLflow tracking URI to the workspace
mlflow.set_tracking_uri(ws.get_mlflow_tracking_uri())

# Create an Azure ML experiment in your workspace
experiment = Experiment(workspace=ws, name='mslearn-diabetes-mlflow')
mlflow.set_experiment(experiment.name)

# start the MLflow experiment
with mlflow.start_run():
    
    print("Starting experiment:", experiment.name)
    
    # Load data
    data = pd.read_csv('data/diabetes.csv')

    # Count the rows and log the result
    row_count = (len(data))
    mlflow.log_metric('observations', row_count)
    print("Run complete")

现在，让我们看看运行期间记录的指标

In [ ]:
# Get the latest run of the experiment
run = list(experiment.get_runs())[0]

# Get logged metrics
print("\nMetrics:")
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))
    
# Get a link to the experiment in Azure ML studio   
experiment_url = experiment.get_portal_url()
print('See details at', experiment_url)

运行上面的代码后，可以使用显示的链接在 Azure 机器学习工作室中查看试验。然后选择该试验的最新运行并查看其“**指标**”选项卡以查看记录的指标。

### 在试验脚本中使用 MLflow

你还可以使用 MLflow 跟踪试验脚本中的指标。

运行以下两个单元格，为使用 MLflow 的试验创建文件夹和脚本。

In [ ]:
import os, shutil

# Create a folder for the experiment files
folder_name = 'mlflow-experiment-files'
experiment_folder = './' + folder_name
os.makedirs(folder_name, exist_ok=True)

# Copy the data file into the experiment folder
shutil.copy('data/diabetes.csv', os.path.join(folder_name, "diabetes.csv"))

In [ ]:
%%writefile $folder_name/mlflow_diabetes.py
from azureml.core import Run
import pandas as pd
import mlflow


# start the MLflow experiment
with mlflow.start_run():
       
    # Load data
    data = pd.read_csv('diabetes.csv')

    # Count the rows and log the result
    row_count = (len(data))
    print('observations:', row_count)
    mlflow.log_metric('observations', row_count)

在 Azure ML 试验脚本中使用 MLflow 跟踪时，开始试验运行时将自动设置 MLflow 跟踪 URI。但是，要在其中运行脚本的环境必须包括所需的“**mlflow**”包。

In [ ]:
from azureml.core import Experiment, ScriptRunConfig, Environment
from azureml.widgets import RunDetails


# Create a Python environment for the experiment (from a .yml file)
env = Environment.from_conda_specification("experiment_env", "environment.yml")

# Create a script config
script_mlflow = ScriptRunConfig(source_directory=experiment_folder,
                                script='mlflow_diabetes.py',
                                environment=env) 

# submit the experiment
experiment = Experiment(workspace=ws, name='mslearn-diabetes-mlflow')
run = experiment.submit(config=script_mlflow)
RunDetails(run).show()
run.wait_for_completion()

与往常一样，你可以在试验运行完成时从中获取记录的指标。

In [ ]:
# Get logged metrics
metrics = run.get_metrics()
for key in metrics.keys():
        print(key, metrics.get(key))

> **详细信息**：若要了解有关运行试验的详细信息，请参阅 Azure ML 文档中的[本主题](https://docs.microsoft.com/azure/machine-learning/how-to-manage-runs)。有关如何在运行期间记录指标的详细信息，请参阅[本主题](https://docs.microsoft.com/azure/machine-learning/how-to-track-experiments)。有关将 Azure ML 试验与 MLflow 集成的更多信息，请参见[本话题](https://docs.microsoft.com/en-us/azure/machine-learning/how-to-use-mlflow)。